In [73]:
from __future__ import print_function
from __future__ import division

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, precision_recall_fscore_support, average_precision_score
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.feature_selection import f_regression, f_classif
from sklearn import svm
from scipy.stats import pearsonr

In [76]:
dev_visual_features = np.load('../data/dev_uploaded_data/features.npy')
dev_annotations = np.load('../data/dev_uploaded_data/valenceArousalAnnotations.npy')
# ravel() here change a one-column vector to an array... i'm sorry, kuang
dev_annotations_fear = np.load('../data/dev_uploaded_data/fearAnnotations.npy').ravel()
dev_visual_features = preprocessing.minmax_scale(dev_visual_features, feature_range=(0, 1), axis=0)
print(dev_visual_features.shape)

variance_selection = VarianceThreshold(threshold=0.01)
bestk_selection_0 = SelectKBest(f_regression, k=100)
bestk_selection_1 = SelectKBest(f_regression, k=100)
bestk_selection_fear = SelectKBest(f_classif, k=100)

dev_visual_features_selected = variance_selection.fit_transform(dev_visual_features)
print(dev_visual_features_selected.shape)
dev_visual_features_selected_0 = bestk_selection_0.fit_transform(dev_visual_features_selected, dev_annotations[:,0])
dev_visual_features_selected_1 = bestk_selection_1.fit_transform(dev_visual_features_selected, dev_annotations[:,1])
dev_visual_features_selected_fear = bestk_selection_fear.fit_transform(dev_visual_features_selected, dev_annotations_fear)

test_visual_features = np.load('../data/test_uploaded_data/features.npy')
test_annotations = np.load('../data/test_uploaded_data/valenceArousalAnnotations.npy')
test_annotations_fear = np.load('../data/test_uploaded_data/fearAnnotations.npy').ravel()
test_visual_features = preprocessing.minmax_scale(test_visual_features, feature_range=(0, 1), axis=0)

test_visual_features_selected = variance_selection.transform(test_visual_features)
test_visual_features_selected_0 = bestk_selection_0.transform(test_visual_features_selected)
test_visual_features_selected_1 = bestk_selection_1.transform(test_visual_features_selected)
test_visual_features_selected_fear = bestk_selection_fear.transform(test_visual_features_selected)

(5274, 1271)
(5274, 647)


In [42]:
# decision tree for task1
model = DecisionTreeRegressor(max_depth=None, min_samples_split=2, random_state=0)
model.fit(dev_visual_features_selected_0, dev_annotations[:,0])
dt_pred_0 = model.predict(test_visual_features_selected_0)
model.fit(dev_visual_features_selected_1, dev_annotations[:,1])
dt_pred_1 = model.predict(test_visual_features_selected_1)

In [43]:
print(mean_squared_error(test_annotations[:,0], dt_pred_0), pearsonr(test_annotations[:,0], dt_pred_0))
print(mean_squared_error(test_annotations[:,1], dt_pred_1), pearsonr(test_annotations[:,1], dt_pred_1))

0.181803164128 (0.038671295421262068, 0.0034708632846477678)
0.14576998356 (0.10025378891999739, 3.1232004544929717e-14)


In [68]:
# random forest for task1
model = RandomForestRegressor(n_estimators=10, criterion='mse',
                              max_depth=None, min_samples_split=2,
                              min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                              max_features='auto', max_leaf_nodes=None)

In [69]:
model.fit(dev_visual_features_selected_0, dev_annotations[:,0])
rf_pred_0 = model.predict(test_visual_features_selected_0)
model.fit(dev_visual_features_selected_1, dev_annotations[:,1])
rf_pred_1 = model.predict(test_visual_features_selected_1)

In [70]:
print(mean_squared_error(test_annotations[:,0], rf_pred_0), pearsonr(test_annotations[:,0], rf_pred_0))
print(mean_squared_error(test_annotations[:,1], rf_pred_1), pearsonr(test_annotations[:,1], rf_pred_1))

0.0989397557421 (0.10990558930043863, 8.195127457107299e-17)
0.0990826732291 (0.012963784869572366, 0.32736798066495776)


In [64]:
# svm for task1
model = svm.SVR()
model.fit(dev_visual_features_selected_0, dev_annotations[:,0]) 
svm_pred_0 = model.predict(test_visual_features_selected_0)
model.fit(dev_visual_features_selected_1, dev_annotations[:,1])
svm_pred_1 = model.predict(test_visual_features_selected_1)

In [65]:
print(mean_squared_error(test_annotations[:,0], svm_pred_0), pearsonr(test_annotations[:,0], svm_pred_0))
print(mean_squared_error(test_annotations[:,1], svm_pred_1), pearsonr(test_annotations[:,1], svm_pred_1))

0.094374676285 (0.13236958205386584, 9.7155106228132347e-24)
0.0897409898834 (0.016610408342657819, 0.20948977050046971)


In [104]:
# random forest for task1=2
model = RandomForestClassifier()
model.fit(dev_visual_features_selected_fear, dev_annotations_fear)
rf_pred_fear = model.predict(test_visual_features_selected_fear)

In [105]:
print(accuracy_score(test_annotations_fear, rf_pred_fear))
print(precision_recall_fscore_support(test_annotations_fear, rf_pred_fear))
print(average_precision_score(test_annotations_fear, rf_pred_fear))

0.959719789842
(array([ 0.96410978,  0.        ]), array([ 0.99527788,  0.        ]), array([ 0.97944593,  0.        ]), array([5506,  204]))
0.0357267950963


In [106]:
# random forest for task1=2
model = svm.SVC()
model.fit(dev_visual_features_selected_fear, dev_annotations_fear)
svm_pred_fear = model.predict(test_visual_features_selected_fear)

In [107]:
print(accuracy_score(test_annotations_fear, svm_pred_fear))
print(precision_recall_fscore_support(test_annotations_fear, svm_pred_fear))
print(average_precision_score(test_annotations_fear, svm_pred_fear))

0.964273204904
(array([ 0.9642732,  0.       ]), array([ 1.,  0.]), array([ 0.9818117,  0.       ]), array([5506,  204]))
0.0357267950963


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
